In [1]:

import pandas as pd
import sqlite3
import os
import plotly.express as px

# --- Configuration ---
PROJECT_ROOT = os.path.abspath(os.path.join(os.getcwd(), '..'))
DATABASE_FILE = os.path.join(PROJECT_ROOT, "garmin_data.db")

# --- Connect and Query ---
con = sqlite3.connect(DATABASE_FILE)
df_activities = pd.read_sql_query("""
    SELECT 
        date(start_time_gmt) as activity_date,
        SUM(distance_m) / 1000 as total_distance_km
    FROM activities
    WHERE sport = 'running'
    GROUP BY activity_date
""", con)
con.close()

# --- Visualize the Prototype ---
# This plot will show up right here in the notebook
fig = px.bar(df_activities, 
             x='activity_date', 
             y='total_distance_km', 
             title='Daily Running Distance')
fig.show()

# You can look at the data directly
print(df_activities.head())

  activity_date  total_distance_km
0    2023-09-14           14.16299
1    2023-09-15           20.86147
2    2023-09-17            9.99602
3    2023-09-19           14.10011
4    2023-09-20           21.84284


# Difficulty analysis
zones Quentin POTTIER
Vitesse Maximale Aérobie (VMA) : 22,3 km/h.
• VO2 max : 76,39 ml/min/kg, excellente performance aérobie.
• Premier seuil ventilatoire (SV1) : 16,5 km/h avec une fréquence cardiaque de 175
bpm.
• Deuxième seuil ventilatoire (SV2) : 19,7 km/h avec une fréquence cardiaque de
194 bpm.
• Fréquence cardiaque maximale atteinte : 204 bpm.
Zones d'intensité : 
Intensité 1 : 
- < 13km/h
- FC = 0 - 153 bpm
Intensité 2 : 
-  13 - 16km/h
- FC = 153 - 173 bpm
Intensité 3 : 
-  17 - 10km/h
- FC = 174 - 188 bpm
Intensité 4 : 
-  19 - 21km/h
- FC = 188 - 1 bpm
Intensité 5 : 
-  21 - 23km/h
- FC = 196 - 204 bpm

In [7]:
import pandas as pd
import sqlite3
import os

# --- Configuration ---
PROJECT_ROOT = os.path.abspath(os.path.join(os.getcwd(), '..'))
DATABASE_FILE = os.path.join(PROJECT_ROOT, "garmin_data.db")

# --- Connect and Query ---
con = sqlite3.connect(DATABASE_FILE)

# --- 1. Define Your Personal HR Zones and Multipliers ---
# Bins define the upper limit of each zone.
hr_bins = [0, 153, 173, 188, 195, 204]
# Labels are the zone numbers.
hr_labels = [1, 2, 3, 4, 5]
# Multipliers for the zTRIMP calculation.
zone_multipliers = {1: 1, 2: 2, 3: 3, 4: 4, 5: 5}

# --- 2. Load All Record Data from the Database ---
con = sqlite3.connect(DATABASE_FILE)
# Only select records that have a heart rate to avoid errors and improve performance.
df_records = pd.read_sql_query("SELECT activity_id, heart_rate FROM records WHERE heart_rate IS NOT NULL", con)
con.close()

# --- 3. Classify Each Record into an HR Zone ---
# The pd.cut function is perfect for this. It efficiently groups the heart_rate data into the bins we defined.
df_records['hr_zone'] = pd.cut(df_records['heart_rate'], bins=hr_bins, labels=hr_labels, right=True, include_lowest=True)

# --- 4. Calculate Time Spent in Each Zone per Activity ---
# First, group by activity and zone, then count the number of records (which equals seconds).
# .unstack(fill_value=0) pivots the zones into columns, filling any missing zones for an activity with 0.
time_in_zones_seconds = df_records.groupby(['activity_id', 'hr_zone']).size().unstack(fill_value=0)

# Convert the time from seconds to minutes for the formula.
time_in_zones_minutes = time_in_zones_seconds / 60

# --- 5. Calculate the Final zTRIMP Score ---
# Create a copy of the time-in-zone data to work with.
df_training_load = time_in_zones_minutes.copy()

# Initialize the score column.
df_training_load['zTRIMP_score'] = 0

# Loop through the multipliers, calculate the score for each zone, and add it to the total.
for zone, multiplier in zone_multipliers.items():
    # Check if the zone column exists (it might not if no time was ever spent in it).
    if zone in df_training_load.columns:
        df_training_load['zTRIMP_score'] += df_training_load[zone] * multiplier

# --- 6. Display the Final Result with Activity Info ---
# For better context, let's merge this with the main activity info.
con = sqlite3.connect(DATABASE_FILE)
df_activities = pd.read_sql_query("SELECT activity_id, start_time_gmt, sport, distance_m FROM activities", con)
con.close()

# Set activity_id as the index on the activities DataFrame to allow for an easy join.
df_activities.set_index('activity_id', inplace=True)

# Join the training load data with the activity data.
df_final_load = df_activities.join(df_training_load)

# --- Output ---
print("Training Load (zTRIMP) Score per Activity:")
# Display the most relevant columns, sorted by the most recent activities.
print(df_final_load[['start_time_gmt', 'sport', 'distance_m', 'zTRIMP_score']].sort_values(by='start_time_gmt', ascending=False).head(10))

print("\n-----------------------------------------\n")

print("Time in Zones (Minutes) per Activity:")
# To see the underlying data, display the time spent in each zone for the same activities.
print(time_in_zones_minutes.loc[df_final_load.sort_values(by='start_time_gmt', ascending=False).head(10).index])

Training Load (zTRIMP) Score per Activity:
                     start_time_gmt     sport  distance_m  zTRIMP_score
activity_id                                                            
20250928162722  2025-09-28 16:27:22   running    10229.53     57.883333
20250928074559  2025-09-28 07:45:59   running    16431.19    132.116667
20250927101104  2025-09-27 10:11:04   running     1223.87      7.583333
20250927081225  2025-09-27 08:12:25   running     8441.00     59.000000
20250926094139  2025-09-26 09:41:39   running     9567.95     55.916667
20250924103013  2025-09-24 10:30:13  training        0.00     25.300000
20250924101631  2025-09-24 10:16:31   running     1914.24      9.066667
20250923163402  2025-09-23 16:34:02   running    11115.01     84.983333
20250923101047  2025-09-23 10:10:47   running    13095.28     96.816667
20250922161057  2025-09-22 16:10:57   running    15550.70     75.816667

-----------------------------------------

Time in Zones (Minutes) per Activity:
hr_zone    

/var/folders/cy/6bvk0sbx603843zpcpb8_6lh0000gq/T/ipykernel_14147/550062015.py:33: FutureWarning:

The default of observed=False is deprecated and will be changed to True in a future version of pandas. Pass observed=False to retain current behavior or observed=True to adopt the future default and silence this warning.

